In [1]:
import numpy as np
import pymysql
import configparser
import json
import time

In [2]:
ip = "139.199.102.41"
config = configparser.RawConfigParser()
config.readfp(open('./settings.cfg'))

dbname = config.get('jx3bla', 'username')
dbpwd = config.get('jx3bla', 'password')
db = pymysql.connect(ip,dbname,dbpwd,"jx3bla",port=3306,charset='utf8')
cursor = db.cursor()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
edition = "3.5.0"
mapdetail = "25人英雄范阳夜变"

sql = """SELECT * FROM ActorStat WHERE edition>='3.5.0' AND mapdetail='454'"""
cursor.execute(sql)
result = cursor.fetchall()

In [8]:
res = result[0]
print(res)


('斗转星移', '安小逢', '2020-09-20', '454', '3.5.0', '0013b609ff763e8a84b01dbf0f2bef36', "{'effectiveDPSList': [['此去经年', '22', 70311.94393741852, 56836.73272490221, 107169.55223880596, 42976.375, 71303.43250036573, 0, 2, '21'], ['夏目以纪', '21', 69275.75749674055, 57368.39765319426, 79210.27753686036, 0.0, 75359.89387309989, 0, 5, '21'], ['虹光', '8', 66411.74054758801, 28844.80964797914, 141279.9334140436, 0.0, 48343.21678879683, 0, 4, 'X1'], ['蠢蠢哒大白', '21', 66383.60886571056, 36393.65840938722, 112201.14532685035, 0.0, 58671.25929275332, 0, 1, '13'], ['丸酱回来了', '2', 56748.77314211213, 38497.23989569752, 71753.15579227697, 24634.2, 56829.68494407724, 0, 1, '00'], ['苍渊渊', '21', 52076.297262059976, 29852.59191655802, 84970.10804970286, 0.0, 53146.48305028394, 0, 4, '00'], ['今天不吃早餐', '4', 51163.199478487615, 23398.87483702738, 105789.48136142625, 14888.15, 39925.78431502933, 0, 5, '30'], ['咩咩咩叽', '4', 49231.54106910039, 28515.485006518906, 83101.3963573287, 0.0, 43780.75086776695, 0, 2, '33'], ['苗疆小蝴

In [9]:
bossDict = {}

for line in result:
    if line[1] not in bossDict:
        bossDict[line[1]] = []
    stat = json.loads(line[6].replace("'", '"'))
    for record in stat["effectiveDPSList"]:
        bossDict[line[1]].append(record)

In [10]:
bossDict

{'安小逢': [['此去经年',
   '22',
   70311.94393741852,
   56836.73272490221,
   107169.55223880596,
   42976.375,
   71303.43250036573,
   0,
   2,
   '21'],
  ['夏目以纪',
   '21',
   69275.75749674055,
   57368.39765319426,
   79210.27753686036,
   0.0,
   75359.89387309989,
   0,
   5,
   '21'],
  ['虹光',
   '8',
   66411.74054758801,
   28844.80964797914,
   141279.9334140436,
   0.0,
   48343.21678879683,
   0,
   4,
   'X1'],
  ['蠢蠢哒大白',
   '21',
   66383.60886571056,
   36393.65840938722,
   112201.14532685035,
   0.0,
   58671.25929275332,
   0,
   1,
   '13'],
  ['丸酱回来了',
   '2',
   56748.77314211213,
   38497.23989569752,
   71753.15579227697,
   24634.2,
   56829.68494407724,
   0,
   1,
   '00'],
  ['苍渊渊',
   '21',
   52076.297262059976,
   29852.59191655802,
   84970.10804970286,
   0.0,
   53146.48305028394,
   0,
   4,
   '00'],
  ['今天不吃早餐',
   '4',
   51163.199478487615,
   23398.87483702738,
   105789.48136142625,
   14888.15,
   39925.78431502933,
   0,
   5,
   '30'],
  ['咩咩咩叽'

In [25]:
DPSpos = {"周贽": 2,
          "狼牙精锐": 2,
          "厌夜": 2,
          "迟驻": 5,
          "白某": 6,
          "安小逢": 2}

occName = {"0": (0, 0, 0), 
     "1": "少林",
     "2": "万花",
     "4": "纯阳",
     "5": "七秀",
     "3": "天策",
     "8": "藏剑",
     "9": "丐帮",
     "10": "明教",
     "6": "五毒",
     "7": "唐门",
     "21": "苍云",
     "22": "长歌",
     "23": "霸刀",
     "24": "蓬莱",
     "25": "凌雪",
}

for name in bossDict:
    if name == "狼牙精锐":
        continue
    allStat = bossDict[name]
    occDict = {}
    num = 0
    for line in allStat:
        if line[1] not in occDict:
            occDict[line[1]] = []
        if line[DPSpos[name]] > 30000:
            occDict[line[1]].append(line[DPSpos[name]])
            num += 1
    table = []
    for occ in occDict:
        data = np.array(occDict[occ])
        res = [occ, occName[occ], data.shape[0], np.mean(data), np.median(data), np.max(data), np.min(data)]
        table.append(res)
    table.sort(key = lambda x: -x[3])
    print("%s的面板DPS统计，样本数：%d"%(name, num))
    for line in table:
        print(line)

安小逢的面板DPS统计，样本数：3448
['25', '凌雪', 220, 64857.458075554074, 63511.2666905512, 98114.57664233577, 30732.860552763817]
['24', '蓬莱', 129, 60743.71937895148, 59952.41271820449, 87017.64690026954, 30881.681885125185]
['21', '苍云', 252, 59550.274294776405, 58883.29601269429, 95824.10122164049, 33586.820851688695]
['23', '霸刀', 194, 56210.5426674771, 56674.90455098935, 89382.98795180723, 30888.02304737516]
['8', '藏剑', 290, 55066.48937668539, 54427.43411636657, 99863.85238095238, 30626.359393232204]
['9', '丐帮', 130, 53432.98038914053, 51662.32318676201, 95260.02611940299, 30936.062909567496]
['3', '天策', 190, 51924.48544374982, 50074.302728519484, 92249.13694267516, 32389.617157490396]
['4', '纯阳', 369, 50792.699821442046, 50337.650856389984, 94321.03503184713, 30255.09923664122]
['22', '长歌', 72, 50785.99119947736, 49645.389033020496, 83449.60541586073, 32962.506285714284]
['7', '唐门', 380, 50566.7430380284, 48815.35987328984, 93923.3730496454, 30253.09162303665]
['1', '少林', 107, 50519.96334901549, 